In [2]:
import tkinter as tk
from tkinter import ttk, filedialog
import cv2
import numpy as np
from PIL import Image, ImageTk


class DibujoApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Dibujador con Interfaz Gráfica")

        self.root.bind("<Control-z>", lambda event: self.deshacer())

        # canvas en blanco
        self.canvas_img = np.ones((500, 700, 3), dtype=np.uint8) * 255
        self.historial = []
        self.dibujando = False
        self.punto_inicial = (0, 0)
        self.figura = 'Línea'

        # lateral con botones
        control_frame = tk.Frame(root)
        control_frame.pack(side=tk.LEFT, padx=10, pady=10)

        ttk.Label(control_frame, text="Figuras:").pack(pady=5)

        for figura in ["Línea", "Rectángulo", "Círculo"]:
            b = ttk.Button(control_frame, text=figura, width=15,
                          command=lambda f=figura: self.set_figura(f))
            b.pack(pady=2)

        ttk.Button(control_frame, text="Deshacer", width=15, command=self.deshacer).pack(pady=10)
        ttk.Button(control_frame, text="Guardar", width=15, command=self.guardar).pack(pady=2)
        ttk.Button(control_frame, text="Salir", width=15, command=root.destroy).pack(pady=10)

        # area de imagen
        self.imagen_label = tk.Label(root)
        self.imagen_label.pack(side=tk.RIGHT, padx=10, pady=10)
        self.actualizar_imagen()

        # eventos del mouse
        self.imagen_label.bind("<Button-1>", self.mouse_down)
        self.imagen_label.bind("<B1-Motion>", self.mouse_move)
        self.imagen_label.bind("<ButtonRelease-1>", self.mouse_up)

        # eventos con el teclado
        self.root.bind("<Control-z>", lambda event: self.deshacer())
        self.root.bind("<Control-g>", lambda event: self.guardar())

        

    def set_figura(self, figura):
        self.figura = figura

    def deshacer(self):
        if self.historial:
            self.canvas_img = self.historial.pop()
            self.actualizar_imagen()

    def guardar(self):
        archivo = filedialog.asksaveasfilename(defaultextension=".png",
                                                filetypes=[("PNG files", "*.png")])
        if archivo:
            cv2.imwrite(archivo, self.canvas_img)
            print(f"Imagen guardada en: {archivo}")

    def mouse_down(self, event):
        self.dibujando = True
        self.punto_inicial = (event.x, event.y)
        self.historial.append(self.canvas_img.copy())

    def mouse_move(self, event):
        if self.dibujando:
            temp = self.canvas_img.copy()
            self.dibujar_forma(temp, self.punto_inicial, (event.x, event.y))
            self.mostrar(temp)

    def mouse_up(self, event):
        if self.dibujando:
            self.dibujar_forma(self.canvas_img, self.punto_inicial, (event.x, event.y))
            self.dibujando = False
            self.actualizar_imagen()

    def dibujar_forma(self, img, pt1, pt2):
        if self.figura == 'Línea':
            cv2.line(img, pt1, pt2, (0, 0, 0), 2)
        elif self.figura == 'Rectángulo':
            cv2.rectangle(img, pt1, pt2, (255, 0, 0), 2)
        elif self.figura == 'Círculo':
            radio = int(((pt2[0] - pt1[0]) ** 2 + (pt2[1] - pt1[1]) ** 2) ** 0.5)
            cv2.circle(img, pt1, radio, (0, 0, 255), 2)

    def actualizar_imagen(self):
        self.mostrar(self.canvas_img)

    def mostrar(self, img_cv):
        img_rgb = cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(img_rgb)
        self.img_tk = ImageTk.PhotoImage(img_pil)
        self.imagen_label.config(image=self.img_tk)

# Ejecución
root = tk.Tk()
app = DibujoApp(root)
root.mainloop()


Imagen guardada en: C:/Users/franc/Desktop/prueba.png
